In [2]:
import pandas as pd
import numpy as np
import sklearn
from coleta_de_dados import Dataset

/guaxim/Projetos/tosh/ai/anomaly-detection/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from preprocessamento import Preprocessing
pp = Preprocessing()
df,faultydf = pp.normal, pp.anomalos[0]


Filtro Savitzky-Golay aplicado em 9 colunas
Filtro Savitzky-Golay aplicado em 9 colunas

Fazendo Resampling e Interpolate de IMU_10Hz.csv
Buracos gerados pelo alinhamento: 44766
Resampling concluído. Novo shape: (876347, 10)

Fazendo Resampling e Interpolate de IMU_hitting_platform.csv
Buracos gerados pelo alinhamento: 900
Resampling concluído. Novo shape: (14992, 10)


In [4]:
df = df.drop(columns=["time"])
faultydf = faultydf.drop(columns=["time"])
df.columns

Index(['gyroZ', 'gyroX', 'magX', 'magY', 'gyroY', 'magZ', 'accX', 'accY',
       'accZ'],
      dtype='object')

In [5]:
#separar manualmente e sem shuffling para nunca treinar com dados futuros
def _train_test_split(df:pd.DataFrame, trainPer=0.6,valPer=0.05, testPer=0.35):
    assert(trainPer + valPer + testPer <= 1.0)

    size = df.shape[0]

    traindf = df.iloc[:int(size*trainPer)]
    valdf = df.iloc[int(size*trainPer)  : int(size*(valPer + trainPer))  ] 
    testdf = df.iloc[int(size*(valPer+ trainPer)): ] 
    return traindf,valdf,testdf

def __show_splitting(train, val, test,expectedTotal):
    total = train.shape[0] + val.shape[0] + test.shape[0]
    assert(total == expectedTotal)
    
    # Calculate percentages
    pcts = (np.array([[len(train), len(val), len(test)]]) / total) * 100
    
    # FIX: Pass a list of 3 separate strings for the 3 columns
    ddf = pd.DataFrame(
        pcts.round(2),
        columns=["Train %", "Val %", "Test %"] 
    )
    print(ddf)


X_train, X_val,X_test = _train_test_split(df)
__show_splitting(X_train,X_val,X_test, df.shape[0])

_, anom_val,anom_test = _train_test_split(faultydf,trainPer=0,valPer=0.1,testPer=0.9)
__show_splitting(_,anom_val,anom_test, faultydf.shape[0])




   Train %  Val %  Test %
0     60.0    5.0    35.0
   Train %  Val %  Test %
0      0.0   10.0    90.0


In [6]:
def getFixedWindows(df, length, overlap, drop_incomplete=True):
    arr = df.values
    step = length - overlap
    n = arr.shape[0]

    starts = range(0, n - length + 1, step)
    windows = np.stack([arr[s:s+length] for s in starts], axis=0) #semelhante a np.array
    return windows

WINDOW_SIZE, WINDOW_OVERLAP = 60, 10

X_train_w = getFixedWindows(X_train, WINDOW_SIZE, WINDOW_OVERLAP)
X_test_w  = getFixedWindows(X_test,  WINDOW_SIZE, WINDOW_OVERLAP)
X_val_w = getFixedWindows(X_val,  WINDOW_SIZE, WINDOW_OVERLAP)

anom_val_w  = getFixedWindows(anom_val, WINDOW_SIZE, WINDOW_OVERLAP)
anom_test_w  = getFixedWindows(anom_test, WINDOW_SIZE, WINDOW_OVERLAP)

X_train_w.shape, anom_test_w.shape
# matrizes 3d, ainda precisam ser flattened

((10515, 60, 9), (269, 60, 9))

In [7]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import numpy as np


X_train_flat = X_train_w.reshape(X_train_w.shape[0], -1)
X_test_flat  = X_test_w.reshape(X_test_w.shape[0], -1)
X_val_flat = X_val_w.reshape(X_val_w.shape[0], -1)

anom_val_flat = anom_val_w.reshape(anom_val_w.shape[0], -1)
anom_test_flat  = anom_test_w.reshape(anom_test_w.shape[0], -1)


X_train_w.shape, X_train_flat.shape



((10515, 60, 9), (10515, 540))

In [8]:
from sklearn.ensemble import IsolationForest
iso = IsolationForest(
    n_estimators=300,
    contamination="auto",
    bootstrap=False,
    random_state=42
)
#l = pp.run()
#X_t = l[0]

iso.fit(X_train_flat)


,"n_estimators n_estimators: int, default=100The number of base estimators in the ensemble.",300
,"max_samples max_samples: ""auto"", int or float, default=""auto""The number of samples to draw from X to train each base estimator.- If int, then draw `max_samples` samples.- If float, then draw `max_samples * X.shape[0]` samples.- If ""auto"", then `max_samples=min(256, n_samples)`.If max_samples is larger than the number of samples provided,all samples will be used for all trees (no sampling).",'auto'
,"contamination contamination: 'auto' or float, default='auto'The amount of contamination of the data set, i.e. the proportionof outliers in the data set. Used when fitting to define the thresholdon the scores of the samples.- If 'auto', the threshold is determined as in the original paper.- If float, the contamination should be in the range (0, 0.5]... versionchanged:: 0.22 The default value of ``contamination`` changed from 0.1 to ``'auto'``.",'auto'
,"max_features max_features: int or float, default=1.0The number of features to draw from X to train each base estimator.- If int, then draw `max_features` features.- If float, then draw `max(1, int(max_features * n_features_in_))` features.Note: using a float number less than 1.0 or integer less than number offeatures will enable feature subsampling and leads to a longer runtime.",1.0
,"bootstrap bootstrap: bool, default=FalseIf True, individual trees are fit on random subsets of the trainingdata sampled with replacement. If False, sampling without replacementis performed.",False
,"n_jobs n_jobs: int, default=NoneThe number of jobs to run in parallel for :meth:`fit`. ``None`` means 1unless in a :obj:`joblib.parallel_backend` context. ``-1`` means usingall processors. See :term:`Glossary ` for more details.",None
,"random_state random_state: int, RandomState instance or None, default=NoneControls the pseudo-randomness of the selection of the featureand split values for each branching step and each tree in the forest.Pass an int for reproducible results across multiple function calls.See :term:`Glossary `.",42
,"verbose verbose: int, default=0Controls the verbosity of the tree building process.",0
,"warm_start warm_start: bool, default=FalseWhen set to ``True``, reuse the solution of the previous call to fitand add more estimators to the ensemble, otherwise, just fit a wholenew forest. See :term:`the Glossary `... versionadded:: 0.21",False


In [9]:


# Predict anomalies  (1 = normal, -1 = anomaly)
X_labels = iso.predict(X_test_flat)
anom_labels = iso.predict(anom_test_flat)

# Count
X_anom = (X_labels == -1).sum()
anom_count = (anom_labels == -1).sum()

print("=== Isolation Forest Results ===")
print(f"X_test (normal):")
print(f"  Normal:   {(X_labels == 1).sum()}")
print(f"  Anomaly:  {X_anom} ({X_anom / len(X_labels)*100:.2f}%)")

print(f"\nY_test (faulty):")
print(f"  Normal:   {(anom_labels == 1).sum()}")
print(f"  Anomaly:  {anom_count} ({anom_count / len(anom_labels)*100:.2f}%)")

=== Isolation Forest Results ===
X_test (normal):
  Normal:   5133
  Anomaly:  1001 (16.32%)

Y_test (faulty):
  Normal:   212
  Anomaly:  57 (21.19%)


In [10]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import numpy as np

# ---------------------------
# Isolation Forest Trainer
# ---------------------------
from hyperparamTuner import Trainer, Evaluator, HyperParamTuner
class IFTrainer(Trainer):
    def __init__(
        self,
        n_estimators=100,
        max_samples="auto",
        contamination="auto",
        random_state=42
    ):
        self.model = IsolationForest(
            n_estimators=n_estimators,
            contamination=contamination,
            random_state=random_state,
            n_jobs=-1
        )

    def TrainerFit(self, X_train, Y_train=None):
        Xs = X_train
        self.model.fit(Xs)

    def TrainerPred(self, X):
        Xs =X
        # sklearn: 1 = normal, -1 = anomaly
        return (self.model.predict(Xs) == 1).astype(int)


# ---------------------------
# Hyperparameter grids
# ---------------------------

window_grid = {"new_window_size": [6, 20,40], "window_overlap": [2,10,15 ]}  # already windowed
model_grid = {
    "n_estimators": [50, 100],
    "contamination": ["auto"]
}

# ---------------------------
# Hyperparameter tuning
# ---------------------------

tuner = HyperParamTuner(
    windowParams=window_grid,
    modelTrainerParams=model_grid,
    X_train=X_train_flat,
    X_val=X_val_flat,
    X_test=X_test_flat,
    anom_val=anom_val_flat,
    anom_test=anom_test_flat
)

tuner.tune(
    Trainer_factory=IFTrainer,
    evaluator=Evaluator()
)

# ---------------------------
# Show best result
# ---------------------------

print("Best score:", tuner.best_score)
print("Best params:")
print(tuner.best_params)

# Optional: inspect all trials
# tuner.results


Best score: 0.847926267281106
Best params:
{'score': np.float64(0.847926267281106), 'metrics': {'accuracy': np.float64(0.847926267281106)}, 'window_params': {'new_window_size': 40, 'window_overlap': 15}, 'model_params': {'n_estimators': 50, 'contamination': 'auto'}}


In [12]:
from sklearn.mixture import GaussianMixture
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

gmm = GaussianMixture(n_components=2, covariance_type="full", random_state=42)
gmm.fit(X_train_flat)
thr = np.percentile(gmm.score_samples(X_val_flat), 5)
x_pred = (gmm.score_samples(X_test_flat) > thr).astype(int)
a_pred = (gmm.score_samples(anom_test_flat) > thr).astype(int)
print((x_pred==0).sum(), (a_pred==0).sum())

y_true = np.concatenate([np.ones_like(x_pred), np.zeros_like(a_pred)])
y_pred = np.concatenate([x_pred, a_pred])

cm = confusion_matrix(y_true, y_pred)
print("Confusion matrix:\n", cm)
print("\nAccuracy:", accuracy_score(y_true, y_pred))
print("\nClassification report:\n", classification_report(y_true, y_pred, target_names=["anomaly", "normal"]))

338 200
Confusion matrix:
 [[ 200   69]
 [ 338 5796]]

Accuracy: 0.9364360456036233

Classification report:
               precision    recall  f1-score   support

     anomaly       0.37      0.74      0.50       269
      normal       0.99      0.94      0.97      6134

    accuracy                           0.94      6403
   macro avg       0.68      0.84      0.73      6403
weighted avg       0.96      0.94      0.95      6403

